In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.9.1
sys.version_info(major=3, minor=9, micro=7, releaselevel='final', serial=0)
matplotlib 3.5.2
numpy 1.23.0
pandas 1.4.3
sklearn 1.1.1
tensorflow 2.9.1
keras.api._v2.keras 2.9.0


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
# print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

(20640, 8)
(20640,)


In [3]:
from sklearn.model_selection import train_test_split

x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11)
print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [7]:
# metric(算子)使用，主要是reset_states的作用

metric = keras.metrics.MeanSquaredError()
print(metric([5.], [2.]))
print('-'*50)
# metrics具有累加功能，第1个是9，第二个是1，平均是5，(9+1)/2
print(metric([0.], [1.]))
print('-'*50)
print(metric.result())
print('-'*50)

# 不想累加就reset
metric.reset_states()  # 每次epoch需要reset
metric([1.], [3.])
print(metric.result())

tf.Tensor(9.0, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(5.0, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(5.0, shape=(), dtype=float32)
--------------------------------------------------
tf.Tensor(4.0, shape=(), dtype=float32)


In [8]:
# 训练集的样本数
print(len(x_train_scaled))
# 特征数
print(x_train.shape[1:])

11610
(8,)


In [9]:
11610/32  # 每次训练给予的样本数（batch_size默认为32）

362.8125

In [10]:
t= np.arange(6).reshape(1, 2, 1, 3)
print(t)
tf.squeeze(t)  # [2, 3]  # squeeze降维，将维数为1的去除

[[[[0 1 2]]

  [[3 4 5]]]]


<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[0, 1, 2],
       [3, 4, 5]])>

In [12]:
# 随机挑选5个样本，看下特征，标签
idx = np.random.randint(0, 1000, size=5)
print(idx)
print('-'*50)
print(x_train_scaled[idx])
print('-'*50)
print(y_train[idx])  # 随机挑选的5个样本的标签值

[269 649  93 741  14]
--------------------------------------------------
[[ 0.2386013  -1.00944236  0.12091293 -0.06109078  1.39400443 -0.08213305
   0.74929087 -1.14103365]
 [ 2.59001399 -0.84924189  0.85550044 -0.2822698  -0.14203108 -0.03203228
  -0.86970344  0.76147762]
 [ 3.5941466   1.87416616  1.25862327 -0.24266415 -0.84699245 -0.00978029
  -0.72973275  0.61666705]
 [ 0.56560379  0.51246213  0.02513846 -0.29642461  0.16826079 -0.02520107
   1.09455191 -1.22592261]
 [-0.8243033  -0.28854023 -0.09552141 -0.05604838  0.89408974 -0.07176204
   0.31071604 -0.1073858 ]]
--------------------------------------------------
[2.624   3.909   5.00001 1.693   0.723  ]


In [15]:
# squeeze的作用
t=tf.constant([[1],[2],[3]])
print(t)
print('-'*50)
print(tf.squeeze(t,1))

tf.Tensor(
[[1]
 [2]
 [3]], shape=(3, 1), dtype=int32)
--------------------------------------------------
tf.Tensor([1 2 3], shape=(3,), dtype=int32)


In [16]:
# 为了一次epoch之内只打印一个损失
for i in range(10):
    print('helloworld',end='\r')

In [23]:
# 1. batch 遍历训练集 metric
#    1.1 自动求导
# 2. epoch结束 验证集 metric

epochs = 100  # 多少次
batch_size = 32 
steps_per_epoch = len(x_train_scaled) // batch_size  # 每一个epoch拿多少数据
print(steps_per_epoch)
print('-'*50)

optimizer = keras.optimizers.SGD()  # 优化器
metric = keras.metrics.MeanSquaredError()  # 算子

# 随机取数据,取出来32个样本
def random_batch(x, y, batch_size=32):
    idx = np.random.randint(0, len(x), size=batch_size)
    return x[idx], y[idx]  # 返回的是训练集样本和对应的标签值


model = keras.models.Sequential([
    keras.layers.Dense(30, activation='relu',input_shape=x_train.shape[1:]),
    keras.layers.Dense(1),
])

print(model.variables)  # 可以看下这个里边有什么
# model.summary()

# 下面一部分相当于替代了fit函数
for epoch in range(epochs):  # 每一轮epochs训练所有的样本
    metric.reset_states()  # 清空损失
    for step in range(steps_per_epoch):
        # 随机取32个样本
        x_batch, y_batch = random_batch(x_train_scaled, y_train,
                                        batch_size)
        with tf.GradientTape() as tape:
            # 得到预测值
            y_pred = model(x_batch)  # 等价于model.predict
 
            # 删减了值为1的维度,二阶张量变为一阶张量
            y_pred = tf.squeeze(y_pred, 1)
        
            # 计算损失
            loss = keras.losses.mean_squared_error(y_batch, y_pred)
            
            # 计算均方误差，362步，都进行了记录
            metric(y_batch, y_pred)
            
        # 求梯度
        grads = tape.gradient(loss, model.variables)
        
        # 梯度和变量绑定，因为后面apply_gradients要求一个元组
        grads_and_vars = zip(grads, model.variables)
        
        # 更新，通过grads_and_vars，去更新模型的model.variables，也就是更新了w,b
        optimizer.apply_gradients(grads_and_vars)  # 与x.assign_sub是等价的
        
        # 打印
        p="Epoch "+str(epoch)+" train mse:"+str(metric.result().numpy())
        print(p, end='\r')
        
    # 打换行的目的是为了新起一行显示    
    print('')
    
    # 搞了一波训练后，去验证集验证
    y_valid_pred = model(x_valid_scaled)
    
    # 删减了值为1的维度
    # print(y_valid_pred.shape)
    y_valid_pred = tf.squeeze(y_valid_pred, 1)
    # print(y_valid_pred.shape)

    valid_loss = keras.losses.mean_squared_error(y_valid_pred, y_valid)
    print("\t", "valid mse: ", valid_loss.numpy())

362
--------------------------------------------------
[<tf.Variable 'dense_10/kernel:0' shape=(8, 30) dtype=float32, numpy=
array([[ 0.1473479 ,  0.30609068, -0.06238267,  0.27690384, -0.01954189,
         0.00218344, -0.05929837,  0.17745522, -0.37781006,  0.23914292,
        -0.15279552,  0.02345532,  0.17490819,  0.38218418,  0.22592315,
        -0.389326  ,  0.20751539, -0.00635123, -0.375674  , -0.23519933,
        -0.27746147, -0.26278192,  0.37221143, -0.10857138,  0.10127619,
        -0.01570717, -0.11808714, -0.22544436, -0.27825546, -0.23839058],
       [-0.20778577, -0.3746894 , -0.07574204,  0.11028692,  0.363919  ,
        -0.2649784 ,  0.03365356,  0.33100334, -0.23710868,  0.01393718,
        -0.2636447 ,  0.06073716, -0.23027836,  0.2791671 ,  0.35890147,
         0.16431907,  0.16490796,  0.36433777, -0.06634071,  0.14151964,
         0.3857738 ,  0.03018284,  0.2128729 , -0.06075829, -0.29646426,
         0.3143483 ,  0.24898008,  0.35130134,  0.13995102,  0.13525376

Epoch 55 train mse:0.33030415
	 valid mse:  0.34524195950023384
Epoch 56 train mse:0.32399157
	 valid mse:  0.3468681312122751
Epoch 57 train mse:0.32366133
	 valid mse:  0.34197258913523665
Epoch 58 train mse:0.34109473
	 valid mse:  0.3427449861183012
Epoch 59 train mse:0.34382677
	 valid mse:  0.34281246394517945
Epoch 60 train mse:0.34233463
	 valid mse:  0.34200200798088626
Epoch 61 train mse:0.33795622
	 valid mse:  0.3463241773760342
Epoch 62 train mse:0.31686732
	 valid mse:  0.3432772323219214
Epoch 63 train mse:0.33575127
	 valid mse:  0.34024931557571875
Epoch 64 train mse:0.30885506
	 valid mse:  0.3395320792447115
Epoch 65 train mse:0.32117867
	 valid mse:  0.3411094190667224
Epoch 66 train mse:0.32760436
	 valid mse:  0.34697335674855223
Epoch 67 train mse:0.31824502
	 valid mse:  0.33845966493078744
Epoch 68 train mse:0.31832863
	 valid mse:  0.34208732837809697
Epoch 69 train mse:0.32984536
	 valid mse:  0.3352101863223627
Epoch 70 train mse:0.33037477
	 valid mse:  0.3